In [ ]:
!pip install -q findspark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 9.9 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=2c6abe9e0953689941f29d95876ac686d9a1c76a002b3e4bf2567affcd5951f4
  Stored in directory: /root/.cache/pip/wheels/b1/59/a0/a1a0624b5e865fd389919c1a10f53aec9b12195d6747710baf
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession, window
from pyspark import SparkConf
from pyspark.sql.functions import *

In [ ]:
my_conf = SparkConf()

In [ ]:
my_conf.set("Saprk.App.Name","Window_aggregration")
my_conf.set("spark.master","local[*]")

In [ ]:
spark = SparkSession.builder.config(conf=my_conf).getOrCreate()

In [ ]:
inputDF = spark.read.format("csv").option("path","/content/windowdata.csv").load()

In [ ]:
inputDF.show()

+--------------+---+---+-----+---------+
|           _c0|_c1|_c2|  _c3|      _c4|
+--------------+---+---+-----+---------+
|         Spain| 49|  1|   67|   174.72|
|       Germany| 48| 11| 1795|  3309.75|
|     Lithuania| 48|  3|  622|  1598.06|
|       Germany| 49| 12| 1852|  4521.39|
|       Bahrain| 51|  1|   54|   205.74|
|       Iceland| 49|  1|  319|   711.79|
|         India| 51|  5|   95|   276.84|
|     Australia| 50|  2|  133|   387.95|
|         Italy| 49|  1|   -2|    -17.0|
|         India| 49|  5| 1280|   3284.1|
|         Spain| 50|  2|  400|  1049.01|
|United Kingdom| 51|200|28782| 75103.46|
|        Norway| 49|  1| 1730|  1867.98|
|United Kingdom| 48|478|68865|166116.72|
|        France| 51|  5|  847|  1702.87|
|      Portugal| 49|  4|  726|  1844.67|
|         Spain| 48|  1|  400|    620.0|
|         India| 48|  7| 2822|  3147.23|
|       Germany| 50| 15| 1973|  5065.79|
|         Italy| 51|  1|  131|    383.7|
+--------------+---+---+-----+---------+
only showing top

In [ ]:
inputDF2 = inputDF.toDF("country","weeknum","num_of_invoices","total_quantity","invoice_number")

In [ ]:
inputDF2.show()

+--------------+-------+---------------+--------------+--------------+
|       country|weeknum|num_of_invoices|total_quantity|invoice_number|
+--------------+-------+---------------+--------------+--------------+
|         Spain|     49|              1|            67|        174.72|
|       Germany|     48|             11|          1795|       3309.75|
|     Lithuania|     48|              3|           622|       1598.06|
|       Germany|     49|             12|          1852|       4521.39|
|       Bahrain|     51|              1|            54|        205.74|
|       Iceland|     49|              1|           319|        711.79|
|         India|     51|              5|            95|        276.84|
|     Australia|     50|              2|           133|        387.95|
|         Italy|     49|              1|            -2|         -17.0|
|         India|     49|              5|          1280|        3284.1|
|         Spain|     50|              2|           400|       1049.01|
|Unite

In [ ]:
from pyspark.sql import Window

In [ ]:
# Creating a Window for finding the running total of the each country and ordering by weeknum
my_window = Window.partitionBy("country").orderBy("weeknum").rowsBetween(Window.unboundedPreceding,Window.currentRow)

In [ ]:
# Running Total is new column so created a new column using withcolumn and passing the window to that
my_df_updated = inputDF2.withColumn("RunningTotal",sum("invoice_number").over(my_window))

In [ ]:
my_df_updated.show()

+---------------+-------+---------------+--------------+--------------+------------------+
|        country|weeknum|num_of_invoices|total_quantity|invoice_number|      RunningTotal|
+---------------+-------+---------------+--------------+--------------+------------------+
|      Australia|     48|              1|           107|        358.25|            358.25|
|      Australia|     49|              1|           214|         258.9|            617.15|
|      Australia|     50|              2|           133|        387.95|1005.0999999999999|
|        Austria|     50|              2|             3|        257.04|            257.04|
|        Bahrain|     51|              1|            54|        205.74|            205.74|
|        Belgium|     48|              1|           528|         346.1|             346.1|
|        Belgium|     50|              2|           285|        625.16|            971.26|
|        Belgium|     51|              2|           942|        838.65|1809.9099999999999|

In [ ]:
df1 = spark.read.csv("/content/customers.csv",header=True,inferSchema=True)

In [ ]:
# Storing the file in the json format
df1.write.format('json').option("path","/content/customer").save()

In [ ]:
df2 = spark.read.csv("/orders.csv",header=True,inferSchema=True)

In [ ]:
df2.write.format("json").option("path","/content/orders").save()

In [ ]:
#loading the json files
ordersDF = spark.read.format("json").option("path","/part-00000-78f4173b-81d2-4d0f-b06b-555cef289bd7-c000.json").load()

In [ ]:
customerDF = spark.read.format("json").option("path","/part-00000-9d2c7dd5-15e6-4a04-afce-d3f0368ef30e-c000.json").load()

In [ ]:
ordersDF.show()

+-----------------+----------+--------+---------------+
|order_customer_id|order_date|order_id|   order_status|
+-----------------+----------+--------+---------------+
|            11599|   00:00.0|       1|         CLOSED|
|              256|   00:00.0|       2|PENDING_PAYMENT|
|            12111|   00:00.0|       3|       COMPLETE|
|             8827|   00:00.0|       4|         CLOSED|
|            11318|   00:00.0|       5|       COMPLETE|
|             7130|   00:00.0|       6|       COMPLETE|
|             4530|   00:00.0|       7|       COMPLETE|
|             2911|   00:00.0|       8|     PROCESSING|
|             5657|   00:00.0|       9|PENDING_PAYMENT|
|             5648|   00:00.0|      10|PENDING_PAYMENT|
|              918|   00:00.0|      11| PAYMENT_REVIEW|
|             1837|   00:00.0|      12|         CLOSED|
|             9149|   00:00.0|      13|PENDING_PAYMENT|
|             9842|   00:00.0|      14|     PROCESSING|
|             2568|   00:00.0|      15|       CO

In [ ]:
customerDF.show()

+-------------+--------------+--------------+-----------+--------------+-----------------+--------------+--------------------+----------------+
|customer_city|customer_email|customer_fname|customer_id|customer_lname|customer_password|customer_state|     customer_street|customer_zipcode|
+-------------+--------------+--------------+-----------+--------------+-----------------+--------------+--------------------+----------------+
|  Brownsville|     XXXXXXXXX|       Richard|          1|     Hernandez|        XXXXXXXXX|            TX|  6303 Heather Plaza|           78521|
|    Littleton|     XXXXXXXXX|          Mary|          2|       Barrett|        XXXXXXXXX|            CO|9526 Noble Embers...|           80126|
|       Caguas|     XXXXXXXXX|           Ann|          3|         Smith|        XXXXXXXXX|            PR|3422 Blue Pioneer...|             725|
|   San Marcos|     XXXXXXXXX|          Mary|          4|         Jones|        XXXXXXXXX|            CA|  8324 Little Common|          

In [ ]:
JoinedDF = ordersDF.join(customerDF,ordersDF.order_customer_id == customerDF.customer_id, "inner")

In [ ]:
JoinedDF.show()

+-----------------+----------+--------+---------------+-------------+--------------+--------------+-----------+--------------+-----------------+--------------+--------------------+----------------+
|order_customer_id|order_date|order_id|   order_status|customer_city|customer_email|customer_fname|customer_id|customer_lname|customer_password|customer_state|     customer_street|customer_zipcode|
+-----------------+----------+--------+---------------+-------------+--------------+--------------+-----------+--------------+-----------------+--------------+--------------------+----------------+
|            11599|   00:00.0|       1|         CLOSED|      Hickory|     XXXXXXXXX|          Mary|      11599|        Malone|        XXXXXXXXX|            NC|8708 Indian Horse...|           28601|
|              256|   00:00.0|       2|PENDING_PAYMENT|      Chicago|     XXXXXXXXX|         David|        256|     Rodriguez|        XXXXXXXXX|            IL|7605 Tawny Horse ...|           60625|
|         

In [ ]:

my_join = ordersDF.order_customer_id == customerDF.customer_id

my_join_type = "right"

In [ ]:
# Cutomer who never placed any orders
No_order_customerDF = ordersDF.join(customerDF, my_join, my_join_type).sort("order_customer_id")

In [ ]:
No_order_customerDF.show()

+-----------------+----------+--------+------------+-------------------+--------------+--------------+-----------+--------------+-----------------+--------------+--------------------+----------------+
|order_customer_id|order_date|order_id|order_status|      customer_city|customer_email|customer_fname|customer_id|customer_lname|customer_password|customer_state|     customer_street|customer_zipcode|
+-----------------+----------+--------+------------+-------------------+--------------+--------------+-----------+--------------+-----------------+--------------+--------------------+----------------+
|             null|      null|    null|        null|             Denver|     XXXXXXXXX|          Mary|        219|       Harrell|        XXXXXXXXX|            CO|9016 Foggy Robin ...|           80219|
|             null|      null|    null|        null|          Henderson|     XXXXXXXXX|       Matthew|       9060|         Patel|        XXXXXXXXX|            NV|7190 Silver Horse...|           89

In [ ]:
DuplicateDF = spark.read.format("csv").option("header",True).option("inferschema",True).option("path","/content/orders.csv").load()

In [ ]:
DuplicateDF.show()
DuplicateDF.printSchema()

+---------+----------+---------+---------------+
|order_id0|order_date|order_id2|   order_status|
+---------+----------+---------+---------------+
|        1|   00:00.0|    11599|         CLOSED|
|        2|   00:00.0|      256|PENDING_PAYMENT|
|        3|   00:00.0|    12111|       COMPLETE|
|        4|   00:00.0|     8827|         CLOSED|
|        5|   00:00.0|    11318|       COMPLETE|
|        6|   00:00.0|     7130|       COMPLETE|
|        7|   00:00.0|     4530|       COMPLETE|
|        8|   00:00.0|     2911|     PROCESSING|
|        9|   00:00.0|     5657|PENDING_PAYMENT|
|       10|   00:00.0|     5648|PENDING_PAYMENT|
|       11|   00:00.0|      918| PAYMENT_REVIEW|
|       12|   00:00.0|     1837|         CLOSED|
|       13|   00:00.0|     9149|PENDING_PAYMENT|
|       14|   00:00.0|     9842|     PROCESSING|
|       15|   00:00.0|     2568|       COMPLETE|
|       16|   00:00.0|     7276|PENDING_PAYMENT|
|       17|   00:00.0|     2667|       COMPLETE|
|       18|   00:00.

In [ ]:
DuplicateDF.write.format("json").mode("overwrite").option("path","/content/orders").save()

In [ ]:
#when we have duplicate columns while loading we could use the above code to make it as casesensitive
spark.conf.set("spark.sql.caseSensitive","true")

In [ ]:
sample_DF = spark.read.json("/insightslogsPipelineruns.json")

In [ ]:
sample_DF.show()

+-----+------------+--------------------+--------------------+--------------------+-------------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------------------+--------------------+
|Level|    category|       correlationId|                 end|             groupId|        level|   location|       operationName|        pipelineName|          properties|          resourceId|               runId|               start|    status|                tags|                time|
+-----+------------+--------------------+--------------------+--------------------+-------------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------------------+--------------------+
|    4|PipelineRuns|f664e844-9b8d-4ef...|2023-02-16T18:01:...|f664e844-9b8d-4ef...|Informational|northeurope|pl_load_file - Su...|   